<table align="left">
  <td>
    <a href="https://colab.research.google.com/drive/1D62kStjydJvh9Xz4ePYRaqY_hgXMSG7J" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# **Redes neuronales antagonistas**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.datasets import fashion_mnist


In [ ]:
def build_generator():

    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod((28, 28, 1)), activation='tanh'))
    model.add(Reshape((28, 28, 1)))

    noise = Input(shape=(100,))
    img = model(noise)

    return Model(noise, img)

def build_discriminator():

    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))

    img = Input(shape=(28, 28, 1))
    validity = model(img)

    return Model(img, validity)


In [ ]:
def compile_gan():

    #optimizer = Adam(0.0002, 0.5)
    optimizer = tf.keras.optimizers.legacy.Adam(0.0002, 0.5)

    # Construir y compilar el discriminador
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy',
                           optimizer=optimizer,
                           metrics=['accuracy'])

    # Construir el generador
    generator = build_generator()

    # Generador toma ruido como entrada y genera imágenes
    z = Input(shape=(100,))
    img = generator(z)

    # Solo entrenaremos el generador en el modelo combinado
    discriminator.trainable = False

    # Discriminador toma imágenes generadas como entrada y determina validez
    validity = discriminator(img)

    # Modelo combinado (stacked generator and discriminator)
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    return generator, discriminator, combined


In [ ]:
def train_gan(epochs, batch_size=128, save_interval=50):

    (X_train, _), (_, _) = fashion_mnist.load_data()

    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    generator, discriminator, combined = compile_gan()

    for epoch in range(epochs):

        # Entrenar el discriminador
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entrenar el generador
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = combined.train_on_batch(noise, valid)

        print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        if epoch % save_interval == 0:
            save_imgs(epoch, generator)

def save_imgs(epoch, generator):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    plt.show()


In [ ]:
train_gan(epochs=10000, batch_size=32, save_interval=1000)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def generate_and_show_imgs(generator, n_images=25):
    r, c = 5, 5  # Esto generará una cuadrícula de 5x5 de imágenes. Asegúrate de que r*c = n_images
    if n_images != r*c:
        raise ValueError("r*c must be equal to n_images")

    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Re-escala imágenes 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c, figsize=(10, 10))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    plt.show()


In [ ]:
generator, _, _ = compile_gan()  # Solo necesitas el generador aquí, por eso ignoramos el discriminador y el modelo combinado.
train_gan(epochs=10000, batch_size=32, save_interval=1000)

# Genera y muestra 25 imágenes
generate_and_show_imgs(generator)


Output hidden; open in https://colab.research.google.com to view.